### Interaction analysis for HLA genotypes and phenotypes across UK Biobank 

#### Genotypes and phenotypes 

Genotypes and phenotypes used for analysis is from [DeBoever et al. 2018](https://www.nature.com/articles/s41467-018-03910-9).

#### Statistical analysis

We will be using [Interact](https://cran.r-project.org/web/packages/Interact/) package for testing, and the [glinternet](https://cran.r-project.org/web/packages/glinternet/index.html) and [hiernet](https://cran.r-project.org/web/packages/hierNet/index.html) packages for multivariate modeling. 

In [3]:
library('glinternet')
library('hierNet')
library('Interact')

In [5]:
# Load HLA genotypes and phenotypes data 
load('/oak/stanford/groups/mrivas/users/jolivier/repos/hla-assoc/scripts/output/make_dosage_rds/hla_phenotypes.Rda')

In [6]:
objects()

[1] "covars"                    "dosages"                  
[3] "master.phenotype"          "master.phenotype.genotype"
[5] "phenotypes"                "rounded_dosages"

We have a few objects included Dataframes documented by [Guhan](https://med.stanford.edu/rivaslab/People.html):
- `covars`: Age, sex, array, and first 40 PCs for individuals listed.
- `dosages`: Raw allelotype dosages for individuals listed.
- `rounded_dosages` Rounded allelotype dosages (cleaner, but less information)
for individuals listed. Possible rounded dosages = [0, 1, 2].
- `phenotypes`: Subset of master phenotype file that has the phenotypes of
interest for the individuals listed. 1 = unaffected, 2 = affected, -9 = 
missing data of some sort.
- `master.phenotype`: Master phenotype file across all 500,000 individuals.
- `master.phenotype.genotype`: Phenotype and genotype data with HLA alleles across 337,208 individuals.

We will focus on the `master.phenotype.genotype` Dataframe for analysis. 

In [7]:
colnames(master.phenotype.genotype)

[1] "FID"             "IID"             "age.x"           "sex.x"          
   [5] "Array.x"         "PC1.x"           "PC2.x"           "PC3.x"          
   [9] "PC4.x"           "PC5.x"           "PC6.x"           "PC7.x"          
  [13] "PC8.x"           "PC9.x"           "PC10.x"          "PC11.x"         
  [17] "PC12.x"          "PC13.x"          "PC14.x"          "PC15.x"         
  [21] "PC16.x"          "PC17.x"          "PC18.x"          "PC19.x"         
  [25] "PC20.x"          "PC21.x"          "PC22.x"          "PC23.x"         
  [29] "PC24.x"          "PC25.x"          "PC26.x"          "PC27.x"         
  [33] "PC28.x"          "PC29.x"          "PC30.x"          "PC31.x"         
  [37] "PC32.x"          "PC33.x"          "PC34.x"          "PC35.x"         
  [41] "PC36.x"          "PC37.x"          "PC38.x"          "PC39.x"         
  [45] "PC40.x"          "BIN_FC1002306"   "INI25908"        "INI25909"       
  [49] "INI25906"        "INI25907"        "INI25904"        "BIN_FC30003486" 
  [53] "INI25902"        "INI25903"        "INI25900"        "INI25901"       
  [57] "MED1140860954"   "INI4079"         "MED1140866738"   "MED1140872150"  
  [61] "INI3829"         "MED1140884488"   "INI25348"        "INI25349"       
  [65] "INI25838"        "INI25839"        "MED1140869176"   "QT_FC1001697"   
  [69] "INI25832"        "INI25833"        "INI25830"        "INI25831"       
  [73] "INI25344"        "INI25345"        "INI25346"        "INI25347"       
  [77] "INI25905"        "MED1141180212"   "MED1140911658"   "MED1141168554"  
  [81] "MED1140875632"   "MED1141164828"   "BIN_FC50020548"  "MED1140869930"  
  [85] "MED1141164060"   "MED1140881472"   "INI884"          "HC28"           
  [89] "BIN1960"         "HC24"            "HC27"            "HC26"           
  [93] "HC21"            "HC22"            "MED1140881474"   "MED1140864286"  
  [97] "MED1140867818"   "MED1140883968"   "MED1140883748"   "MED1140878304"  
 [101] "MED1141157252"   "MED1140870956"   "MED1140925978"   "MED1140911732"  
 [105] "HC251"           "HC250"           "HC253"           "HC252"          
 [109] "HC255"           "HC254"           "HC257"           "HC256"          
 [113] "HC259"           "MED1141146606"   "INI23210"        "BIN5485"        
 [117] "INI23213"        "INI23214"        "INI23215"        "INI23216"       
 [121] "MED1140852948"   "INI21036"        "INI21037"        "INI21034"       
 [125] "INI21032"        "INI21033"        "INI21030"        "INI21031"       
 [129] "INI21039"        "MED1140868170"   "MED1140878186"   "QT_FC1002080"   
 [133] "INI22402"        "BIN2040"         "MED1141174032"   "HC355"          
 [137] "MED1140861090"   "HC356"           "HC105"           "HC104"          
 [141] "HC107"           "HC106"           "BIN_FC10003393"  "HC100"          
 [145] "HC103"           "HC102"           "INI22154"        "MED1140879854"  
 [149] "HC109"           "MED1141145896"   "MED1140929012"   "HC351"          
 [153] "BIN_FC8006154"   "BIN_FC8006155"   "INI25075"        "INI22404"       
 [157] "INI22405"        "MED1140879798"   "BIN20401"        "QT_FC1004570"   
 [161] "INI25836"        "MED1140871052"   "MED1140865716"   "INI25837"       
 [165] "INI25834"        "BIN_FC10003731"  "INI25835"        "INI20520"       
 [169] "QT_FC1002936"    "MED1141180226"   "MED1140862944"   "INI25009"       
 [173] "INI25008"        "INI25007"        "INI25006"        "INI25005"       
 [177] "INI25004"        "INI25003"        "INI25002"        "INI25001"       
 [181] "INI25000"        "BIN_FC10002463"  "INI30060"        "MED1140882394"  
 [185] "INI5132"         "INI5107"         "INI1160"         "INI5134"        
 [189] "INI5106"         "BIN_FC50020126"  "INI30010"        "BIN_FC10005496" 
 [193] "MED1140867490"   "MED1140928284"   "MED1140882618"   "INI25645"       
 [197] "INI25644"        "INI25647"        "INI25646"        "INI25641"       
 [201] "INI25640"        "INI25643"        "INI25642"

We will test the analysis for [type 1 diabetes](https://en.wikipedia.org/wiki/Diabetes_mellitus_type_1#Genetics), [celiac disease](https://en.wikipedia.org/wiki/Coeliac_disease#Genetics), [ankylosing spondylitis](https://en.wikipedia.org/wiki/Ankylosing_spondylitis#Genetic_testing), and [psoriasis](https://en.wikipedia.org/wiki/Psoriasis#Genetics). 

The codes can be mapped from the latest master phenotype file prepared by Matthew Aguirre in the following [Github link](https://github.com/rivas-lab/ukbb-tools/blob/master/02_phenotyping/icdinfo.txt).

We will begin with type 1 diabetes where the small GWAS can be seen in [Global Biobank Engine](https://biobankengine.stanford.edu/coding/HC337). 

In [8]:
dim(master.phenotype.genotype)

[1] 337208   3061

Select columns for covariates

In [10]:
colnames(master.phenotype.genotype)[c(3,4,seq(6,15,1))]

[1] "age.x"  "sex.x"  "PC1.x"  "PC2.x"  "PC3.x"  "PC4.x"  "PC5.x"  "PC6.x" 
 [9] "PC7.x"  "PC8.x"  "PC9.x"  "PC10.x"

Select columns for HLA alleles

In [11]:
which(colnames(master.phenotype.genotype) == "A_101")

[1] 2655

In [12]:
which(colnames(master.phenotype.genotype) == "DPA1_401")

[1] 3016

In [13]:
which(colnames(master.phenotype.genotype) == "HC337")

[1] 2206

In [14]:
library(ggplot2)

In [23]:
dim(cbind(master.phenotype.genotype[,2655:3016], master.phenotype.genotype[,c(3,4,seq(6,15,1))]))
nrow(cbind(master.phenotype.genotype[,2655:3016],master.phenotype.genotype[,c(3,4,seq(6,15,1))]))

[1] 337208    374

[1] 337208

In [25]:
ncol(cbind(master.phenotype.genotype[,2655:3016],master.phenotype.genotype[,c(3,4,seq(6,15,1))]))

[1] 374

In [27]:
# have to convert phenotypes to 0,1 
master.phenotype.genotype[master.phenotype.genotype[,2206] == 1,2206] <- 0 
master.phenotype.genotype[master.phenotype.genotype[,2206] == 2,2206] <- 1

In [ ]:
fit = glinternet(cbind(master.phenotype.genotype[master.phenotype.genotype[,2206] != -9,2655:3016],master.phenotype.genotype[master.phenotype.genotype[,2206] != -9,c(3,4,seq(6,15,1))]),master.phenotype.genotype[master.phenotype.genotype[,2206] != -9,2206], numLevels = rep(1,374), family = "binomial")


In [30]:
unique(master.phenotype.genotype[,2206])

[1]  0 -9  1